In [6]:
import sqlite3
from datetime import datetime

# Connect to the SQLite database (or create a new one if it doesn't exist)
conn = sqlite3.connect('text_database.db')
cursor = conn.cursor()

text_types = ('личное сообщение', 'статья', 'новость', 'запись в личном блоге', 'публичный комментарий')
# Create the 'texts' table
cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS texts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        text TEXT,
        source TEXT,
        text_type TEXT CHECK(text_type IN {text_types}),
        date TIMESTAMP,
        url TEXT,
        platform TEXT,
        full_text TEXT,
        title TEXT,
        source_references TEXT,
        source_references_present BOOLEAN,
        logical_errors_present BOOLEAN,
        hatespeech_present BOOLEAN,
        hatespeech_indicators TEXT)
''')

# Create the 'manipulation_methods' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS manipulation_methods (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        manipulation_method_name TEXT)
''')

# Create the 'logical_fallacies' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS logical_fallacies (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        logical_fallacy_name TEXT)
''')

# Create the 'text_2_manipulation_methods' junction table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS text_2_manipulation_methods (
        text_id INTEGER,
        manipulation_method_id INTEGER,
        FOREIGN KEY (text_id) REFERENCES texts (id),
        FOREIGN KEY (manipulation_method_id) REFERENCES manipulation_methods (id),
        PRIMARY KEY (text_id, manipulation_method_id)
    )
''')


# Create the 'text_2_logical_fallacies' junction table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS text_2_logical_fallacies (
        text_id INTEGER,
        logical_fallacy_id INTEGER,
        FOREIGN KEY (text_id) REFERENCES texts (id),
        FOREIGN KEY (logical_fallacy_id) REFERENCES manipulation_methods (id),
        PRIMARY KEY (text_id, logical_fallacy_id)
    )
''')


# Example data to insert into the tables
example_data = {
    'text': 'Example texts text',
    'source': 'Example Source',
    'text_type': 'запись в личном блоге',
    'date': datetime.now(),
    'url': 'http://example.com',
    'platform': 'Web',
    'full_text': 'Full texts text',
    'title': 'Example Title',
    'source_references': 'Example source_references',
    'source_references_present': True,
    'logical_errors_present': False,
    'hatespeech_present': False,
    'hatespeech_indicators': 'example here'
}

manipulation_methods = ['Демонизация врага', 'Уловка Галилея']

# Insert the example data into the 'texts' table
cursor.execute('''
    INSERT INTO texts (
        text, source, text_type, date, url, platform, full_text, title, source_references,
        source_references_present, logical_errors_present,
        hatespeech_present, hatespeech_indicators
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', (
    example_data['text'], example_data['source'], example_data['text_type'],
    example_data['date'], example_data['url'], example_data['platform'],
    example_data['full_text'], example_data['title'], example_data['source_references'],
    example_data['source_references_present'], example_data['logical_errors_present'],
    example_data['hatespeech_present'], example_data['hatespeech_indicators']
))

# Insert manipulation methods into the 'manipulation_methods' table
for method in manipulation_methods:
    cursor.execute('INSERT INTO manipulation_methods (manipulation_method_name) VALUES (?)', (method,))

# Get the ID of the inserted texts article
text_id = cursor.lastrowid

# Associate manipulation methods with the texts article in the 'texts_manipulation' table
for method in manipulation_methods:
    cursor.execute('''
        INSERT INTO text_2_manipulation_methods (text_id, manipulation_method_id)
        VALUES (?, (SELECT id FROM manipulation_methods WHERE manipulation_method_name = ?))
    ''', (text_id, method))

# Commit changes and close the connection
conn.commit()
conn.close()